In [1]:
import sys, os

from pybox2d.framework import Framework,Testbed
from pybox2d import *
import pybox2d
print("AAAA")

class Web(Framework):
    name = "Web"
    description = "goo"
    bodies = []
    joints = []

    def __init__(self, gui):
        super(Web, self).__init__(gui)

        print('acc')
        # The ground
        ground = self.world.create_static_body(
            shapes=edge_shape(vertices=[(-40, 0), ( 40, 0)])
        )
        print('b')
        fixture = fixture_def(shape=circle_shape(radius=1),
                               density=5, friction=0.9)
        print('cc')

        self.bodies =[]
        shape = [10, 3]
        for x in range(shape[0]):
            b = []
            for y in range(shape[1]):
                pos = (x*4,y*4 +1 )
                b.append(self.world.create_dynamic_body(position=pos,fixtures=fixture, angular_damping=10.5, linear_damping=0.5))
            self.bodies.append(b)
    #     bodies = self.bodies

        # We will define the positions in the local body coordinates, the length
        # will automatically be set by the __init__ of the b2DistanceJointDef


        self.joints = []
        def connect(body_a, body_b):
            stiffness, damping = linear_stiffness(frequency_hz=1.5, damping_ratio=0.01, body_a=body_a, body_b=body_b)
            dfn = distance_joint_def(
                stiffness=stiffness,
                damping=damping,
                body_a=body_a,
                body_b=body_b,
                # local_anchor_a=local_anchor_a,
                # local_anchor_b=local_anchor_b
            )
            self.joints.append(self.world.create_joint(dfn))

        for x in range(shape[0]):
            for y in range(shape[1]):

                if x+1 < shape[0]:
                    connect(self.bodies[x][y], self.bodies[x+1][y])
                if y+1 < shape[1]:
                    connect(self.bodies[x][y], self.bodies[x][y+1])

                if x+1 < shape[0] and y+1 < shape[1]:
                    connect(self.bodies[x][y], self.bodies[x+1][y+1])

                if x-1 >= 0 and y+1 < shape[1]:
                    connect(self.bodies[x][y], self.bodies[x-1][y+1])
                
                


AAAA


<frozen importlib._bootstrap>:228: FutureWarning: pybind11-bound class 'pybox2d._pybox2d.b2World' is using an old-style placement-new '__init__' which has been deprecated. See the upgrade guide in pybind11's docs. This message is only visible when compiled in debug mode.


In [ ]:
import matplotlib.pyplot as plt

def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

def scale_color(color):
    return [float(c)/255.0 for c in color]

class MatplotlibDebugDraw(pybox2d.DebugDraw):
    def __init__(self, fig, ax):
        self.fig = fig
        self.ax = ax
        super(MatplotlibDebugDraw,self).__init__(float_colors=False)

        print("in debug draw constructor")
    
        #self.ppm = ppm
        #self.ippm = 1.0 / self.ppm
        self.outline_width = 0.01
        self.segment_width = 0.01
        self._bounding_box = [ [0,0],[0,0]]

        self.joint_colors = {
            JointType.unknown_joint :   (230, 25, 75),
            JointType.revolute_joint :  (255,0, 0),
            JointType.prismatic_joint : (255, 225, 25),
            JointType.distance_joint :  (0, 130, 200),
            JointType.pulley_joint :    (245, 130, 48),
            JointType.mouse_joint :     (145, 30, 180),
            JointType.gear_joint :      (70, 240, 240),
            JointType.wheel_joint :     (240, 50, 230),
            JointType.weld_joint :      (210, 245, 60),
            JointType.friction_joint :  (250, 190, 190),
            # JointType.rope_joint :      (0, 128, 128),
            JointType.motor_joint :     (230, 190, 255),
        }

    def reset_bounding_box(self):
        self._bounding_box = [ [0,0],[0,0]]

    def _update_bounding_box(self, p):
        for c in range(2):
            if p[c] < self._bounding_box[0][c]:
                self._bounding_box[0][c] = p[c]

            if p[c] > self._bounding_box[1][c]:
                self._bounding_box[1][c] = p[c]

    def draw_solid_circle(self, center, radius, axis, color):
        circle = plt.Circle(center, radius, color=scale_color(color))
        self.ax.add_patch(circle)
        pass
                
    def draw_circle(self, center, radius, color):
        # circle = plt.Circle(center, radius, color=scale_color(color))
        # self.ax.add_patch(circle)
        pass

    def draw_segment(self,v1, v2, color):
        # print("draw_segment")
        pass
            
    def draw_polygon(self,vertices, color):
        #print("draw_solid_circle")
        pass

    def draw_solid_polygon(self,vertices, color):
        # print("draw_solid_polygon")
        pass

    def draw_particles(self, centers, radius, colors=None):
        # print("draw_particles")    
        pass
    def draw_joint(self, joint):
        # print("draw_joint")
        pass



import IPython
import time

class MatplotlibTestbedGui(object):
    def __init__(self, testbed):
        fig, ax = plt.subplots(figsize=(20,20)) # note we must use plt.subplots, not plt.subplot
        self.fig = fig
        self.ax = ax
        self.testbed = testbed
        self.framework = self.testbed.exampleCls(gui=self)
        self.framework.gui = self
        self.debug_draw = MatplotlibDebugDraw(fig=fig,ax=ax)
        flags = ['shape','joint','aabb','pair','center_of_mass','particle']
        self.debug_draw.clear_flags(flags)
        for flag in flags:
            self.debug_draw.append_flags(flag)
        self.world = self.framework.world
        self.world.set_debug_draw(self.debug_draw)
    

        
    def run(self):
        self.fig.canvas.draw()
        for i in range(2000):
            # print("step")
            self.framework.step(0.05)
            self.world.draw_debug_data()
            self.ax.plot()
            IPython.display.display(self.fig)
            time.sleep(0.001)
            
            IPython.display.clear_output(wait=True)
            self.ax.clear()
            # for p in self.ax.patches:
            #     p.remove()


testbed = Testbed(guiType=MatplotlibTestbedGui)
testbed.setExample(Web)
testbed.run()